# Step 1: Load and Inspect the Data
The goal here is to load the CSV file and view its contents. We can inspect the data to understand its structure.

In [8]:
import pandas as pd

# Load the dataset
file_path = "/content/TRADES_CopyTr_90D_ROI.csv"  # Update with your file path
df = pd.read_csv(file_path)

# Display the first few rows of the dataset to inspect its structure
print(df.head())


              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...


# Step 2: Handle and Clean the Data
The dataset contains a Trade_History column that stores information in a more complex format (as a string representation of a list of dictionaries). We need to convert this into a more usable format.

In [9]:
import ast

# Define a simple function to safely convert the 'Trade_History' column
def convert_trade_history(trade_data):
    try:
        return ast.literal_eval(trade_data)  # Converts string to a list of dictionaries
    except (ValueError, SyntaxError):
        return None  # In case the data is invalid, return None

# Apply this function to each entry in 'Trade_History'
df['Trade_History'] = df['Trade_History'].apply(convert_trade_history)

# Remove rows with invalid 'Trade_History'
df_cleaned = df.dropna(subset=['Trade_History'])

# Display the cleaned data
print(df_cleaned.head())


              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...


# Explanation:
**ast.literal_eval:** Safely converts a string representation of a list/dictionary back into a Python object (list of dictionaries here).\
**apply:** This function is used to apply the conversion to each row of the column.\
**dropna:** Removes rows where the Trade_History could not be converted.


# Step 3: Extract Useful Trade Information
Now we want to pull out specific details from each trade, such as time, symbol, side (BUY/SELL), price, quantity, and profit.

In [14]:
# Create a function to extract relevant details from 'Trade_History'
def extract_trades(trade_history):
    trades_list = []
    for trade in trade_history:
        # Extract important fields from each trade
        trades_list.append({
            'symbol': trade['symbol'],
            'side': trade['side'],
            'price': trade['price'],
            'realizedProfit': trade.get('realizedProfit', 0)  # Default to 0 if not available
        })
    return trades_list

# Apply this function to the trade history for each account using .loc
df_cleaned.loc[:, 'Extracted_Trades'] = df_cleaned['Trade_History'].apply(extract_trades)

# Display the extracted trade information
print(df_cleaned[['Port_IDs', 'Extracted_Trades']].head())


              Port_IDs                                   Extracted_Trades
0  3925368433214965504  [{'symbol': 'SOLUSDT', 'side': 'BUY', 'price':...
1  4002413037164645377  [{'symbol': 'NEARUSDT', 'side': 'SELL', 'price...
2  3923766029921022977  [{'symbol': 'ETHUSDT', 'side': 'SELL', 'price'...
3  3994879592543698688  [{'symbol': 'ETHUSDT', 'side': 'SELL', 'price'...
4  3926423286576838657  [{'symbol': 'ETHUSDT', 'side': 'SELL', 'price'...


# Explanation:
**extract_trades:** This function loops through each trade in the Trade_History and pulls out relevant fields.\
**get('realizedProfit', 0):** Safely retrieves the profit for each trade, defaulting to 0 if missing.

# step 4: Calculate Metrics for Each Account
Now that we have clean trade data, we can calculate metrics such as Win Rate, Total Profit (PnL), and ROI.

In [15]:
# Function to calculate Win Rate, Total Profit (PnL), and ROI
def calculate_metrics(trades):
    total_positions = len(trades)
    win_positions = sum(1 for trade in trades if trade['realizedProfit'] > 0)  # Profitable trades
    total_profit = sum(trade['realizedProfit'] for trade in trades)
    total_investment = sum(trade['price'] for trade in trades)

    win_rate = win_positions / total_positions if total_positions > 0 else 0
    roi = (total_profit / total_investment) * 100 if total_investment > 0 else 0

    return pd.Series({
        'Win Rate': win_rate,
        'Total Profit (PnL)': total_profit,
        'ROI': roi
    })

# Apply the function to calculate metrics for each account using .loc
df_cleaned[['Win Rate', 'Total Profit (PnL)', 'ROI']] = df_cleaned['Extracted_Trades'].apply(calculate_metrics)

# Display the metrics
print(df_cleaned[['Port_IDs', 'Win Rate', 'Total Profit (PnL)', 'ROI']].head())


              Port_IDs  Win Rate  Total Profit (PnL)       ROI
0  3925368433214965504  0.333333         6789.436739  0.055715
1  4002413037164645377  0.185764         3686.969761  0.512650
2  3923766029921022977  0.285147          772.729032  0.024090
3  3994879592543698688  0.552833         3658.765062  0.644991
4  3926423286576838657  0.341004         1309.459960  0.025232


<ipython-input-15-58d4c9196ecb>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[['Win Rate', 'Total Profit (PnL)', 'ROI']] = df_cleaned['Extracted_Trades'].apply(calculate_metrics)


# Step 5: Rank and Select Top 20 Accounts
Finally, we rank the accounts based on their metrics, such as ROI or Total Profit, and select the top 20 accounts.

In [13]:
# Rank accounts based on ROI using .loc to avoid SettingWithCopyWarning
df_cleaned.loc[:, 'Rank'] = df_cleaned['ROI'].rank(ascending=False)

# Sort accounts by ROI and get the top 20
top_20 = df_cleaned.sort_values(by='ROI', ascending=False).head(20)

# Display the top 20 accounts
print(top_20[['Port_IDs', 'ROI', 'Win Rate', 'Total Profit (PnL)']])

# Save the top 20 accounts to a CSV file
top_20.to_csv('top_20_accounts.csv', index=False)


                Port_IDs           ROI  Win Rate  Total Profit (PnL)
46   3977234346014419201  42334.836545  0.397590         2427.292831
44   4023697433751327232  40611.681855  0.095238         2408.224000
41   3983074113875692800  38138.783342  0.325581         2424.829520
96   4004713168329653760  37699.211283  0.074074         1667.198610
45   4029506971304830209  33222.767603  0.057692         2413.654000
104  4030708912786799105  18475.320743  0.033333         1612.843770
18   4023697881318718465  14737.259902  0.085714         3151.950620
110  4029507714735307777   6704.062891  0.037736         1537.899960
21   3891020560590657281   1489.410444  0.647597         2856.300564
109  4039279455324236544    704.239548  0.553517         1038.807419
31   4034786679304751616    522.937085  0.383178         2521.075869
93   4035034554469394176    179.198828  0.167089         2517.320800
106  4030392204221219329    151.402901  0.094118         1347.758010
127  3999240873283311617    108.76

# Explanation:
**rank: **Assigns a rank to each account based on its ROI (or you can choose any other metric).\
**sort_values:** Sorts the DataFrame by the selected metric (ROI in this case).\
**head(20):** Selects the top 20 accounts.